In [1]:
# goals
# is winrate dependent on time spent in last game?
# is winrate dependent on accuracy?
# is winrate dependent on accuracy of game before, if this game is a reque?
# is winrate dependent on # of reques?

# how to find time spent in a game?
# how do you determine if a game is a reque?

import archives_manager
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.width', 10000)

In [2]:
start_unix = 1696176000 # oct 1st '23
end_unix = 1701190800 # nov 28th '23
player_name = "BIG_TONKA_T"

games_archive = archives_manager.get_games_between_timestamps(
    player_name=player_name,
    start_unix=start_unix,
    end_unix=end_unix,
    time_class='rapid',
    filter_func=archives_manager.build_archive_filter(rated=True)
)

archives_manager.simplified_archived_game(games_archive[0])

{'url': 'https://www.chess.com/game/live/89923795267',
 'end_time': 1696185098,
 'date': '2023.10.01',
 'rated': True,
 'time_class': 'rapid',
 'white': {'username': 'Quarini', 'rating': 1063, 'result': 'resigned'},
 'black': {'username': 'BIG_TONKA_T', 'rating': 1090, 'result': 'win'}}

In [3]:
archives_manager.simplified_archived_game(games_archive[-1])

{'url': 'https://www.chess.com/game/live/94630695929',
 'end_time': 1700892464,
 'date': '2023.11.25',
 'rated': True,
 'time_class': 'rapid',
 'white': {'username': 'cgbs1013', 'rating': 1434, 'result': 'resigned'},
 'black': {'username': 'BIG_TONKA_T', 'rating': 1408, 'result': 'win'}}

In [4]:
games_data = []

unix_range = end_unix - start_unix

for game in games_archive:
    opp_games_archive = None
    opp_name = archives_manager.get_opponent_name(game, player_name)

    # log progress
    print(opp_name, round((game['end_time'] - start_unix) / (unix_range), 2))

    try:
        opp_games_archive = archives_manager.get_games_between_timestamps(
            player_name=opp_name,
            start_unix=game['end_time'] - 30 * 24 * 60 * 60,
            end_unix=game['end_time'],
            time_class='rapid',
            filter_func=archives_manager.build_archive_filter(rated=True)
        )
    except archives_manager.ArchiveRetrievalError as e:
        print(f'{opp_name} archive failed')

    if opp_games_archive is None:
        continue

    games_data.append({
        'game': game,
        'opp_name': opp_name,
        'opp_games_archive': opp_games_archive
    })

Quarini 0.0
hgarciarod 0.0
HERFJERF 0.0
olas109 0.0
InkLv 0.0
alamrini 0.0
djgreen1 0.0
Pytoo91 0.0
mumergaddafi 0.0
Copertus 0.0
CyborgCerberus94 0.0
Nadirmitch 0.0
Rensa0 0.0
wimperneel 0.0
nightjumper39 0.01
Spiderbreeze 0.01
milton2000 0.01
ahmedtaha09 0.01
Godofspeeds 0.01
Krop_duster 0.01
nickringgod 0.01
8akk3 0.01
lilchernobyl 0.01
SamtheTimTamMan 0.01
unicorndash1 0.01
Robari1 0.01
DrimoDramo 0.01
Hoaiarc 0.01
ochuckster123 0.01
SirKennard 0.01
Rick_woote 0.01
Callsign_Gimli 0.01
zzyix 0.01
RedscareRAM 0.01
Alex-Sokol 0.01
ejonokuchi 0.01
EasyRider3202 0.01
instant_10 0.01
Ysulai 0.01
JohannFehrmann 0.01
Ek3498 0.01
georgemetcalf22 0.01
Rotorhead64 0.01
defeatlazydoor 0.01
karthikpan 0.01
OrangPeeler 0.01
jackpotbrandon 0.01
Pseudonym20 0.01
GeorgeLamponi 0.01
aleksandrstrilnyk 0.01
Deejai1220 0.01
morethancipher 0.01
ClarkGriswald 0.01
ToplessChess11 0.02
gokselur 0.02
yellowmoose 0.02
P3rs0Nwastaken 0.02
nikhil6hama 0.02
Wickrama 0.02
gardemarine 0.02
Ametaka 0.02
sauciers 0

In [5]:
tyler_rows = []
opp_data = []

for data in games_data:
    game = data['game']
    opp_name = data['opp_name']
    opp_games_archive = data['opp_games_archive']

    # analyze the opponent's history (for opp_data)
    opp_game_rows = []
    for opp_game in opp_games_archive:
        # get elo of the 'Player' from the perspective of player_name=opp_name, to get that historical elo rating
        opp_game_elo = archives_manager.get_elo(opp_game, player_name=opp_name)

        opp_game_rows.append({
            'unix': opp_game['end_time'],
            'elo': opp_game_elo['Player'], # 'Player' elo from the perspective of player_name=opp_name
            'elo_diff': opp_game_elo['Player'] - opp_game_elo['Opponent'], # Difference between opponent's rating, and opponent's oppenent rating
            # time of day as 'cos_24h' and 'sin_24h'
            # player pieces left
            # opp pieces left
            # move count
            # accuracy
            # acc_diff
            # average eval change per move, per game state
            # opening classification
            # average time spent per move, per game state
            'color': archives_manager.get_color(opp_game, opp_name),
            'won': archives_manager.get_won(opp_game, opp_name)
        })
    opp_data.append(opp_game_rows)

    # analyze this game (for tyler_rows)
    elo = archives_manager.get_elo(game, player_name)
    acc = archives_manager.get_accuracy(game, player_name)
    color = archives_manager.get_color(game, player_name)

    tyler_rows.append({
        # basic info
        'url': game['url'],
        'unix': game['end_time'],
        'player_elo': elo['Player'],
        'opp_elo': elo['Opponent'],
        'opp_name': opp_name,

        # predictors
        'opp_30d': len(opp_games_archive),
        'elo_diff': elo['Player'] - elo['Opponent'],
        'color': color,
        # time of day as 'cos_24h', 'sin_24h' https://chat.openai.com/share/fb073052-f000-4160-bace-fdee65678589

        # game review
        'player_acc': None if acc is None else acc['Player'],
        'opp_acc': None if acc is None else acc['Opponent'],
        'acc_diff': None if acc is None else acc['Player'] - acc['Opponent'],
        # player pieces left
        # opp pieces left
        # move count
        # average eval change per move, per game state
        # average time spent per move, per game state
        # opening classification https://www.nature.com/articles/s41598-023-31658-w.pdf

        # targets
        'won': archives_manager.get_won(game, player_name)
    })

In [6]:
# for each opponent history data, fix the elo since we know the true value based on the associated game in the player history

# this output highlights the issue
print(tyler_rows[0]) # tyler's least recent game (beginning of player data)
print(opp_data[0][-1]) # for that game, the opponent's most recent game in history is the same game but with guessed elo

{'url': 'https://www.chess.com/game/live/89923795267', 'unix': 1696185098, 'player_elo': 1090, 'opp_elo': 1063, 'opp_name': 'Quarini', 'opp_30d': 249, 'elo_diff': 27, 'color': False, 'player_acc': 69.43, 'opp_acc': 62.25, 'acc_diff': 7.180000000000007, 'won': 1}
{'unix': 1696185098, 'elo': 1063, 'elo_diff': -27, 'color': True, 'won': 0}


In [7]:
for i in range(len(tyler_rows)):
    opp_elo = tyler_rows[i]['opp_elo']
    opp_data[i][-1]['elo'] = opp_elo

print(tyler_rows[0]) # tyler's least recent game (beginning of player data)
print(opp_data[0][-1]) # for that game, the opponent's most recent game in history is the same game but with guessed elo

{'url': 'https://www.chess.com/game/live/89923795267', 'unix': 1696185098, 'player_elo': 1090, 'opp_elo': 1063, 'opp_name': 'Quarini', 'opp_30d': 249, 'elo_diff': 27, 'color': False, 'player_acc': 69.43, 'opp_acc': 62.25, 'acc_diff': 7.180000000000007, 'won': 1}
{'unix': 1696185098, 'elo': 1063, 'elo_diff': -27, 'color': True, 'won': 0}


In [15]:
def append_reque_columns(df):
    num_reque = []
    time_since_prev = []

    prev_unix = None
    consecutive_requests = 0

    for index, row in df.iterrows():
        current_unix = row['unix']
        
        if prev_unix is None:
            time_since_prev.append(0)
        else:
            time_since_prev.append(current_unix - prev_unix)

        if prev_unix is None or (current_unix - prev_unix) > 1250:
            consecutive_requests = 0
        else:
            consecutive_requests += 1
        
        num_reque.append(consecutive_requests)
        
        prev_unix = current_unix

    df['num_reque'] = num_reque
    df['time_since_prev'] = time_since_prev

In [24]:
# turn tyler_rows into tyler_df and make even more predictors
tyler_df = pd.DataFrame(tyler_rows)

append_reque_columns(tyler_df)

# ma5, ma20, x-ma5, x-ma20, d_x-ma5, d_x-ma20
# is_reque
# num_reque
# prev_avg_eval_change _open _mid _end
# prev_time_spent _open _mid _end
# prev_acc
# prev_acc_diff
# prev_ player opp _pieces_left


# finding reques
# https://chat.openai.com/share/dc34a56e-11a7-4e53-844a-b98258651090
# is_reque = [num_reque != 0]

opp_rows = []

for index, row in tyler_df.iterrows():
    this_opp_data = opp_data[index]

    if this_opp_data[-1]['unix'] != row['unix']:
        opp_rows.append({})
    else:
        opp_df = pd.DataFrame(this_opp_data)
        append_reque_columns(opp_df)
        opp_rows.append({
            'opp_num_reque': opp_df.iloc[-1]['num_reque'], # trying to access last item in this dataframe
            'opp_time_since_prev': opp_df.iloc[-1]['time_since_prev']
        })

opp_df = pd.DataFrame(opp_rows)
tyler_df = pd.concat([tyler_df, opp_df], axis=1)

tyler_df['opp_num_reque'] = tyler_df['opp_num_reque'].astype('Int64')
tyler_df['opp_time_since_prev'] = tyler_df['opp_time_since_prev'].astype('Int64')

tyler_df.head(25)

,url,unix,player_elo,opp_elo,opp_name,opp_30d,elo_diff,color,player_acc,opp_acc,acc_diff,won,num_reque,time_since_prev,opp_num_reque,opp_time_since_prev
0,https://www.chess.com/game/live/89923795267,1696185098,1090,1063,Quarini,249,27,False,69.43,62.25,7.18,1.0,0,0,2,325
1,https://www.chess.com/game/live/89924356301,1696186352,1097,1091,hgarciarod,339,6,True,78.20,69.40,8.80,1.0,0,1254,2,1108
2,https://www.chess.com/game/live/89926164039,1696187659,1105,1139,HERFJERF,113,-34,False,92.66,80.43,12.23,1.0,0,1307,0,48092
3,https://www.chess.com/game/live/89926822767,1696188843,1114,1091,olas109,178,23,True,67.25,60.95,6.30,NaN,1,1184,8,977
4,https://www.chess.com/game/live/89927951455,1696189372,1113,1105,InkLv,116,8,False,83.82,72.41,11.41,1.0,2,529,1,557
5,https://www.chess.com/game/live/89928581889,1696190619,1121,1105,alamrini,308,16,True,80.57,78.02,2.55,0.0,3,1247,0,5208
6,https://www.chess.com/game/live/89929743189,1696191447,1113,1143,djgreen1,193,-30,True,64.50,57.93,6.57,1.0,4,828,3,832
7,https://www.chess.com/game/live/89931020153,1696192994,1122,1088,Pytoo91,221,34,False,51.35,64.64,-13.29,0.0,0,1547,0,1318
8,https://www.chess.com/game/live/89932135549,1696193859,1113,1078,mumergaddafi,121,35,True,73.48,58.18,15.30,1.0,1,865,18,892
9,https://www.chess.com/game/live/89935195289,1696196998,1120,1122,Copertus,505,-2,True,79.47,66.51,12.96,1.0,0,3139,0,13265


In [28]:
tyler_df.drop(['url'], axis=1).to_csv('tyler_opp_num_reque.csv', index=False)

In [ ]:
# for each opp_rows in opp_data, turn opp_rows into opp_df and make simliar predictors

# ma5, ma20, x-ma5, x-ma20, d_x-ma5, d_x-ma20
# is_reque
# num_reque
# prev_avg_eval_change _open _mid _
# prev_time_spent _open _mid _end
# prev_acc
# prev_acc_diff
# prev_ player opp _pieces_left

In [ ]:
# ideas for classifying/clustering players/games/opponents

# # # cluster board states
# 1. extract features of the board
# white/black _pieces, white/black _passed_pawns, open/semi-open-white/semi-open-black/closed _files, developed_pieces, material_advantage, engine_eval
# some features for pinned material
# features for important spots for material, 'knights in the center' 'rooks on the 7th'
# features for pieces defending other pieces
# features for board control of certain areas (mid, left flank, right flank, top, bottom) that belong to each color, by placement and by areas to move
# 2. do PCA on the feature space of boards across many games for a given player
# look at the most interesting/variable features by the scores in the first few principal componenets
#

# do PCA on the features of board state